In [1]:
using JuMP
using Ipopt
using Complementarity
using NamedArrays

In [49]:
T = [:a,:b,:c,:d]

tech = NamedArray([2 2;5 2; 7 4; 10 1e99],(T,[:cost,:cap]))

c = tech[:,:cost]
k = tech[:,:cap]

4-element Named Vector{Float64}
A  │ 
───┼───────
a  │    2.0
b  │    2.0
c  │    4.0
d  │ 1.0e99

In [50]:
nlp_model = Model(Ipopt.Optimizer)

@variables(nlp_model,begin
    P>=0
    PS>=0
    CS>=0
    S>=0 #I believe S is demand, or D
    0<=Q[t=T]<=k[t]
end)

@objective(nlp_model,Max, CS+PS)

@constraints(nlp_model,begin
    price, P == 10-S*10/6
    supply, S == sum(Q[t] for t∈T)
    psurplus, PS == sum((P-c[t])*Q[t] for t∈T)
    csurplus, CS == (10-P) * S/2
end)

optimize!(nlp_model)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:       24
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        8
                     variables with only lower bounds:        5
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.9999980e-02 9.97e+00 1.78e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [51]:
for var in all_variables(nlp_model)
    println("$var -> $(value(var))")
end

P -> 5.000000000420054
PS -> 6.000000123739848
CS -> 7.499999998739806
S -> 2.9999999997479683
Q[a] -> 2.0000000191643164
Q[b] -> 0.9999999988289247
Q[c] -> -8.746564487890712e-9
Q[d] -> -9.498708252884754e-9


In [53]:
mcp_model = MCPModel()

@variables(mcp_model,begin
    Q[t=T]>=0
    RK[t=T]>=0
    D
    P
end)

@mapping(mcp_model, profit[t=T],
    RK[t]+c[t] - P
)

@mapping(mcp_model, capacity[t=T],
    k[t] - Q[t]
)

@mapping(mcp_model, demand, 
    D - (6*(1-P/10))
)

@mapping(mcp_model, market,
    sum(Q[t] for t∈T) - D
)

@complementarity(mcp_model,profit,Q)
@complementarity(mcp_model,capacity,RK)
@complementarity(mcp_model,demand,D)
@complementarity(mcp_model,market,P)

solveMCP(mcp_model)


Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.8868e+20             0.0e+00 (demand)
    1     1     2     1 6.0000e+19  1.0e+00    0.0e+00 (market)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 6.0000e+19           I 0.0e+00 6.0e+19 (market)
    1     2     3     3 8.1980e+03  1.0e+00 SO 0.0e+00 8.2e+03 (demand)
    2     4     4     4 4.4409e-16  1.0e+00 SO 0.0e+00 4.4e-16 (demand)

Major Iterations. . . . 2
Minor Iterations. . . . 6
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 4.440892e-16


:Solved

In [54]:
print(generate_report(mcp_model))

var_name	 value		 margin
Q[a]		 2.0		 0.0
Q[b]		 1.0		 0.0
Q[c]		 0.0		 2.0
Q[d]		 0.0		 5.0
RK[a]		 3.0		 0.0
RK[b]		 0.0		 1.0
RK[c]		 0.0		 4.0
RK[d]		 0.0		 1.0e99
D		 3.0		 -0.0
P		 5.0		 0.0


In [55]:
for var in all_variables(nlp_model)
    println("$var -> $(value(var))")
end

P -> 5.000000000420054
PS -> 6.000000123739848
CS -> 7.499999998739806
S -> 2.9999999997479683
Q[a] -> 2.0000000191643164
Q[b] -> 0.9999999988289247
Q[c] -> -8.746564487890712e-9
Q[d] -> -9.498708252884754e-9


In [56]:
for var in all_variables(mcp_model)
    println("$var -> $(result_value(var))")
end

Q[a] -> 2.0
Q[b] -> 0.9999999999999996
Q[c] -> 0.0
Q[d] -> 0.0
RK[a] -> 3.0
RK[b] -> 0.0
RK[c] -> 0.0
RK[d] -> 0.0
D -> 2.9999999999999996
P -> 5.0


In [57]:
@assert all(isapprox.(value.(nlp_model[:Q]),result_value.(mcp_model[:Q]);atol = 1e-7)) "Q variable solutions do not match"

@assert value(nlp_model[:P]) ≈ result_value(mcp_model[:P]) "P does not match"

@assert value(nlp_model[:S]) ≈ result_value(mcp_model[:D]) "Demand doesn't match"

@assert result_value.(mcp_model[:RK]) .≈ [3,0,0,0] "RK doesn't match"

println("GAMS gives RK as all zeros")